In [ ]:
BASE     = '/path/to/data/processed/grid_part_1.pkl'
PRICE    = '/path/to/data/processed/grid_part_2.pkl'
CALENDAR = '/path/to/data/processed/grid_part_3.pkl'

In [14]:
import numpy as np
import pandas as pd
df = pd.concat([pd.read_pickle(BASE),
                    pd.read_pickle(PRICE).iloc[:,2:],
                    pd.read_pickle(CALENDAR).iloc[:,2:]],
                    axis=1)
df['d'] = df['d'].str.replace('d_', '', regex=False).astype(np.int16)

In [15]:
df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,release,wm_yr_wk,...,event_type_2,snap_CA,snap_TX,snap_WI,tm_d,tm_m,tm_y,tm_wm,tm_dw,tm_w_end
0,CA_1_HOBBIES_1_008,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,1,12.0,0,11101,...,NaN,0,0,0,29,1,0,5,5,1
1,CA_1_HOBBIES_1_009,HOBBIES_1_009,HOBBIES_1,HOBBIES,CA_1,CA,1,2.0,0,11101,...,NaN,0,0,0,29,1,0,5,5,1
2,CA_1_HOBBIES_1_010,HOBBIES_1_010,HOBBIES_1,HOBBIES,CA_1,CA,1,0.0,0,11101,...,NaN,0,0,0,29,1,0,5,5,1
3,CA_1_HOBBIES_1_012,HOBBIES_1_012,HOBBIES_1,HOBBIES,CA_1,CA,1,0.0,0,11101,...,NaN,0,0,0,29,1,0,5,5,1
4,CA_1_HOBBIES_1_015,HOBBIES_1_015,HOBBIES_1,HOBBIES,CA_1,CA,1,4.0,0,11101,...,NaN,0,0,0,29,1,0,5,5,1


In [ ]:
import pandas as pd

df_wide = pd.read_csv("/path/to/data/sales_test_evaluation.csv")

In [17]:
df_long = df_wide.melt(
    id_vars=["item_id", "dept_id", "cat_id", "store_id", "state_id"],
    var_name="d",          # This will hold the day column names like 'd_1942'
    value_name="sales"     # This will hold the sales values
)

In [18]:
df_long["d"] = (
    df_long["d"]
    .str.replace("d_", "", regex=False)  # Remove the 'd_' part
    .astype(np.int16)                         # Convert to integer
)

In [21]:
df_long["id"] = df_long["store_id"]  + "_" + df_long["item_id"] 
#df_long["id"] = df_long["id"].astype("category")

In [22]:
df_long = df_long[
    ["id", "item_id", "dept_id", "cat_id", "store_id", "state_id", "d", "sales"]
]

In [23]:
df_long.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales
0,CA_1_HOBBIES_1_001,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,1942,2
1,CA_1_HOBBIES_1_002,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,1942,0
2,CA_1_HOBBIES_1_003,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,1942,0
3,CA_1_HOBBIES_1_004,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,1942,0
4,CA_1_HOBBIES_1_005,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,1942,2


In [24]:
print("df dtypes:\n", df[["id", "d"]].dtypes)
print("df_long dtypes:\n", df_long[["id", "d"]].dtypes)

print("Sample df keys:\n", df[["id", "d"]].head())
print("Sample df_long keys:\n", df_long[["id", "d"]].head())

df dtypes:
 id    category
d        int16
dtype: object
df_long dtypes:
 id    object
d      int16
dtype: object
Sample df keys:
                    id  d
0  CA_1_HOBBIES_1_008  1
1  CA_1_HOBBIES_1_009  1
2  CA_1_HOBBIES_1_010  1
3  CA_1_HOBBIES_1_012  1
4  CA_1_HOBBIES_1_015  1
Sample df_long keys:
                    id     d
0  CA_1_HOBBIES_1_001  1942
1  CA_1_HOBBIES_1_002  1942
2  CA_1_HOBBIES_1_003  1942
3  CA_1_HOBBIES_1_004  1942
4  CA_1_HOBBIES_1_005  1942


In [25]:
import pandas as pd

# Merge the original df with df_long on ["id", "d"]
df_merged = df.merge(
    df_long[["id", "d", "sales"]], 
    on=["id", "d"], 
    how="left",          # Keep all rows from df (the "left" DataFrame)
    suffixes=("", "_new")  # So the new sales column becomes "sales_new"
)

# Now df_merged has:
#   "id", "item_id", "store_id", "d", "sales", "sales_new"
#
# - "sales" is the original df's sales
# - "sales_new" is the matching sales from df_long (NaN if no match)

In [26]:
df_merged.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,release,wm_yr_wk,...,snap_CA,snap_TX,snap_WI,tm_d,tm_m,tm_y,tm_wm,tm_dw,tm_w_end,sales_new
0,CA_1_HOBBIES_1_008,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,1,12.0,0,11101,...,0,0,0,29,1,0,5,5,1,NaN
1,CA_1_HOBBIES_1_009,HOBBIES_1_009,HOBBIES_1,HOBBIES,CA_1,CA,1,2.0,0,11101,...,0,0,0,29,1,0,5,5,1,NaN
2,CA_1_HOBBIES_1_010,HOBBIES_1_010,HOBBIES_1,HOBBIES,CA_1,CA,1,0.0,0,11101,...,0,0,0,29,1,0,5,5,1,NaN
3,CA_1_HOBBIES_1_012,HOBBIES_1_012,HOBBIES_1,HOBBIES,CA_1,CA,1,0.0,0,11101,...,0,0,0,29,1,0,5,5,1,NaN
4,CA_1_HOBBIES_1_015,HOBBIES_1_015,HOBBIES_1,HOBBIES,CA_1,CA,1,4.0,0,11101,...,0,0,0,29,1,0,5,5,1,NaN


In [27]:
df_merged.tail()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,release,wm_yr_wk,...,snap_CA,snap_TX,snap_WI,tm_d,tm_m,tm_y,tm_wm,tm_dw,tm_w_end,sales_new
47735392,WI_3_FOODS_3_823,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,1969,NaN,0,11621,...,0,0,0,19,6,5,3,6,1,0.0
47735393,WI_3_FOODS_3_824,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,1969,NaN,0,11621,...,0,0,0,19,6,5,3,6,1,0.0
47735394,WI_3_FOODS_3_825,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,1969,NaN,0,11621,...,0,0,0,19,6,5,3,6,1,0.0
47735395,WI_3_FOODS_3_826,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,1969,NaN,230,11621,...,0,0,0,19,6,5,3,6,1,5.0
47735396,WI_3_FOODS_3_827,FOODS_3_827,FOODS_3,FOODS,WI_3,WI,1969,NaN,304,11621,...,0,0,0,19,6,5,3,6,1,1.0


In [28]:
df_merged["sales"] = df_merged["sales_new"].where(
    df_merged["sales_new"].notna(),  # condition
    df_merged["sales"]              # value if condition is False
)
df_merged.drop(columns=["sales_new"], inplace=True)

In [29]:
start_date = pd.to_datetime("2011-01-29")

# Create the 'date' column by adding (d - 1) days to the start date
df_merged['date'] = start_date + pd.to_timedelta(df_merged['d'] - 1, unit='D')


In [30]:
df_merged = df_merged.sort_values(['id', 'date'])
df_merged.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,release,wm_yr_wk,...,snap_CA,snap_TX,snap_WI,tm_d,tm_m,tm_y,tm_wm,tm_dw,tm_w_end,date
576,CA_1_FOODS_1_001,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,1,3.0,0,11101,...,0,0,0,29,1,0,5,5,1,2011-01-29
11508,CA_1_FOODS_1_001,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2,0.0,0,11101,...,0,0,0,30,1,0,5,6,1,2011-01-30
22440,CA_1_FOODS_1_001,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,3,0.0,0,11101,...,0,0,0,31,1,0,5,0,0,2011-01-31
33372,CA_1_FOODS_1_001,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,4,1.0,0,11101,...,1,1,0,1,2,0,1,1,0,2011-02-01
44304,CA_1_FOODS_1_001,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,5,4.0,0,11101,...,1,0,1,2,2,0,1,2,0,2011-02-02


In [31]:
df_merged.tail()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,release,wm_yr_wk,...,snap_CA,snap_TX,snap_WI,tm_d,tm_m,tm_y,tm_wm,tm_dw,tm_w_end,date
47611999,WI_3_HOUSEHOLD_2_516,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,1965,0.0,0,11620,...,0,1,1,15,6,5,3,2,0,2016-06-15
47642489,WI_3_HOUSEHOLD_2_516,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,1966,1.0,0,11620,...,0,0,0,16,6,5,3,3,0,2016-06-16
47672979,WI_3_HOUSEHOLD_2_516,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,1967,0.0,0,11620,...,0,0,0,17,6,5,3,4,0,2016-06-17
47703469,WI_3_HOUSEHOLD_2_516,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,1968,0.0,0,11621,...,0,0,0,18,6,5,3,5,1,2016-06-18
47733959,WI_3_HOUSEHOLD_2_516,HOUSEHOLD_2_516,HOUSEHOLD_2,HOUSEHOLD,WI_3,WI,1969,0.0,0,11621,...,0,0,0,19,6,5,3,6,1,2016-06-19


In [ ]:
import pickle

# Specify the file path where you want to save the dictionary
file_path = "/path/to/pklFiles/df_master.pkl"

# Save the dictionary to a .pkl file
with open(file_path, "wb") as file:
    pickle.dump(df_merged, file)

print(f"Dictionary saved as {file_path}")

Dictionary saved as /home/karavellas/Desktop/Thesis/Υλοποίηση/pklFiles/df_master.pkl
